In [1]:
import datetime as dt

import pandas as pd
import numpy as np
from lenskit.algorithms import item_knn, user_knn
from sklearn.model_selection import ParameterGrid

from recsys4daos.datasets import to_lenskit
from recsys4daos.model_selection import cvtt_open, explore_hparams
from recsys4daos.evaluation import test_with_hparams_lenskit

import paths

/home/daviddavo/recsys4daos/src/recsys4daos/model_selection.py:12: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


# Parameters

In [2]:
# Dataset config
ORG_NAME = 'Decentraland'
SPLITS_FREQ = 'W-THU'  # Split weekly
LAST_FOLDS = 20  # Use just last 10 splits
SPLITS_NORMALIZE = True

# Evaluation
K_RECOMMENDATIONS: list[int] = [1,3,5,10,15,100]
SPLITS_FREQ: str = 'W-THU' # split weekly
SPLITS_NORMALIZE = True # Wether or not to move everything to 00:00
LAST_FOLDS = 10 # Use just last 10 splits
LAST_FOLD_DATE_STR: str = None

# Search space config
WINDOW_SIZES = ['7d', '14d', '21d', '30d', '60d', '90d', '10YE']
ITEMKNN_Ks = [1,2,3,4,5,6,7,8,9,10,15]

OPTIM_METRIC = 'map@10'

In [3]:
# Parameters
EXECUTION_ID = "2024-09-04T10:00"
ORG_NAME = "Lido"
SPLITS_FREQ = "W-WED"
LAST_FOLDS = 10
SPLITS_NORMALIZE = True
LAST_FOLD_DATE_STR = "2022-07-06"


# Load the dataset

In [4]:
dfp = paths.load_proposals(ORG_NAME)
dfv = paths.load_votes(ORG_NAME)

print(dfp.info())
print(dfv.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 223 entries, 0 to 222
Data columns (total 6 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   id                 223 non-null    object        
 1   author             223 non-null    object        
 2   date               223 non-null    datetime64[us]
 3   start              223 non-null    datetime64[us]
 4   end                223 non-null    datetime64[us]
 5   platform_proposal  223 non-null    object        
dtypes: datetime64[us](3), object(3)
memory usage: 10.6+ KB
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 56473 entries, 0 to 56472
Data columns (total 4 columns):
 #   Column    Non-Null Count  Dtype         
---  ------    --------------  -----         
 0   id        56473 non-null  object        
 1   proposal  56473 non-null  object        
 2   voter     56473 non-null  object        
 3   date      56473 non-null  datetime64[us]
dtypes:

In [5]:
df = to_lenskit(dfv)
df

,user,item,timestamp,rating
0,0x68335b3ac272c8238b722963368f87de736b64d6,cda99ddd-8555-5c25-ba90-433bc3495f45,2021-02-02 07:26:45,1
1,0xc0afef712a1341ca78da145b2afad346e8c3574f,cda99ddd-8555-5c25-ba90-433bc3495f45,2021-02-02 09:26:08,1
2,0x11099ac9cc097d0c9759635b8e16c6a91ecc43da,cda99ddd-8555-5c25-ba90-433bc3495f45,2021-02-02 10:43:28,1
3,0xdcddb68e99aac82b649ac15b82530c2b7f079e9f,cda99ddd-8555-5c25-ba90-433bc3495f45,2021-02-02 15:58:10,1
4,0x10b05dd4cf5decf597bbd831a3c4bc2f9bf51fb0,cda99ddd-8555-5c25-ba90-433bc3495f45,2021-02-02 16:12:08,1
...,...,...,...,...
56468,0x7821f8b55d58d508ced1dd4ce031f0985ce1e46a,df19148b-6dd0-55b0-abb3-eb172d7b7cdb,2023-06-29 17:58:25,1
56469,0xa094e287128d62e6e872daba552ac139bae71354,a136bcda-9842-52c9-a070-4922625067d0,2023-06-29 17:58:37,1
56470,0xa094e287128d62e6e872daba552ac139bae71354,df19148b-6dd0-55b0-abb3-eb172d7b7cdb,2023-06-29 17:59:00,1
56471,0xf7aa189442b2d5cc4846a363892f15ff8b15f41c,3e2ac59f-c78f-57d5-b442-744af4ff6549,2023-06-29 17:59:13,1


## Split in folds

In [6]:
all_folds = { f.end:f for f in cvtt_open(
    df, SPLITS_FREQ, dfp.reset_index(), remove_not_in_train_col='item', col_item='item', last_fold=LAST_FOLD_DATE_STR,
)}
last_folds_idx = list(all_folds.keys())[-LAST_FOLDS:]
last_folds_idx

[Timestamp('2022-05-04 00:00:00'),
 Timestamp('2022-05-11 00:00:00'),
 Timestamp('2022-05-18 00:00:00'),
 Timestamp('2022-05-25 00:00:00'),
 Timestamp('2022-06-01 00:00:00'),
 Timestamp('2022-06-08 00:00:00'),
 Timestamp('2022-06-15 00:00:00'),
 Timestamp('2022-06-22 00:00:00'),
 Timestamp('2022-06-29 00:00:00'),
 Timestamp('2022-07-06 00:00:00')]

# Item-based KNN

In [7]:
def testHParamsItemKNN(fold, k: int, window_size=None):
    # Create algorithm
    algo = item_knn.ItemItem(
        feedback='implicit', # VERY IMPORTANT
        min_sim=0,
        # min_nbrs=0,
        nnbrs=k,  # the maximum number of neighbors for scoring each item (None for unlimited)
    )
    return test_with_hparams_lenskit(algo, all_folds[fold], K_RECOMMENDATIONS, window_size)

pd.Series(testHParamsItemKNN(last_folds_idx[-1],5,'14d'))

/home/daviddavo/recsys4daos/.direnv/python-3.11/lib/python3.11/site-packages/numba/np/ufunc/parallel.py:371: NumbaWarning: The TBB threading layer requires TBB version 2021 update 6 or later i.e., TBB_INTERFACE_VERSION >= 12060. Found TBB_INTERFACE_VERSION = 12050. The TBB threading layer is disabled.
  warnings.warn(problem)
Numba is using threading layer omp - consider TBB


found 1 potential runtime problems - see https://boi.st/lkpy-perf


/home/daviddavo/recsys4daos/.direnv/python-3.11/lib/python3.11/site-packages/lenskit/algorithms/item_knn.py:119: NumbaTypeSafetyWarning: unsafe cast from uint64 to int64. Precision may be lost.
  b = blocks[bi]


fold_t             2022-07-06 00:00:00
time_train                    5.579756
time_rec                       0.22907
open_proposals                       3
min_recs                             1
avg_recs                      2.411321
precision@1                   0.841509
precision@3                   0.545912
precision@5                   0.327547
precision@10                  0.163774
precision@15                  0.109182
precision@100                 0.016377
ndcg@1                        0.390543
ndcg@3                        0.431553
ndcg@5                        0.431553
ndcg@10                       0.431553
ndcg@15                       0.431553
ndcg@100                      0.431553
map@1                         0.841509
map@3                         0.905031
map@5                         0.905031
map@10                        0.905031
map@15                        0.905031
map@100                       0.905031
recall@1                      0.571698
recall@3                 

## Exploring hparams

In [8]:
results = explore_hparams(
    testHParamsItemKNN, 
    ParameterGrid({
    'fold': last_folds_idx,
    'k': ITEMKNN_Ks, # ,20,25,30,35,40,45,50,60,70,80,90,100],
    'window_size': WINDOW_SIZES,
    }),
    paths.hparams_progress('itemknn', ORG_NAME, SPLITS_FREQ, SPLITS_NORMALIZE),
)
mdfi = pd.DataFrame(results)
mdfi

Restored checkpoint from ../.cache/Lido/hparams-itemknn_W-WED_normalize.pkl with 847 results


  0%|          | 0/770 [00:00<?, ?it/s]

,fold,k,window_size,fold_t,time_train,time_rec,open_proposals,min_recs,avg_recs,precision@1,...,recall@10,recall@15,recall@100,r-precision@1,r-precision@3,r-precision@5,r-precision@10,r-precision@15,r-precision@100,time_eval
0,2022-05-04,1,7d,2022-05-04,0.000882,0.001641,1,NaN,NaN,0.000000,...,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.070738
1,2022-05-04,1,14d,2022-05-04,0.000936,0.002844,1,1.0,1.000000,1.000000,...,1.0,1.0,1.0,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.123234
2,2022-05-04,1,21d,2022-05-04,0.000983,0.165014,1,1.0,1.000000,1.000000,...,1.0,1.0,1.0,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.127073
3,2022-05-04,1,30d,2022-05-04,0.001770,0.033063,1,1.0,1.000000,1.000000,...,1.0,1.0,1.0,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.126675
4,2022-05-04,1,60d,2022-05-04,0.001263,0.034857,1,1.0,1.000000,1.000000,...,1.0,1.0,1.0,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.125980
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
765,2022-07-06,15,21d,2022-07-06,0.001546,0.260791,3,1.0,2.463333,0.876667,...,1.0,1.0,1.0,0.683777,0.990602,0.990602,0.990602,0.990602,0.990602,0.177595
766,2022-07-06,15,30d,2022-07-06,0.005510,0.283008,3,1.0,2.468750,0.806250,...,1.0,1.0,1.0,0.691214,0.949458,0.949458,0.949458,0.949458,0.949458,0.178213
767,2022-07-06,15,60d,2022-07-06,0.002860,0.313606,3,1.0,2.311047,0.755814,...,1.0,1.0,1.0,0.739103,0.912420,0.912420,0.912420,0.912420,0.912420,0.180416
768,2022-07-06,15,90d,2022-07-06,0.003273,0.334395,3,1.0,2.348901,0.774725,...,1.0,1.0,1.0,0.748818,0.915430,0.915430,0.915430,0.915430,0.915430,0.182368


### Best overall hparams

In [9]:
display_columns = ['time_train', 'avg_recs'] + [ c for c in mdfi.columns if c.endswith('@5') or c.endswith('@10') or c.endswith('@100') ]
overall_hparams = mdfi[mdfi['fold'] > last_folds_idx[0]].groupby(['window_size', 'k']).mean().sort_values(OPTIM_METRIC, ascending=False)
overall_hparams[display_columns]

time_train  avg_recs  precision@5  precision@10  \
window_size k                                                     
90d         15    0.002945  2.121412     0.245339      0.122670   
            10    0.003320  2.121412     0.245339      0.122670   
            8     0.003426  2.121412     0.245339      0.122670   
            9     0.003416  2.121412     0.245339      0.122670   
            7     0.002751  2.121412     0.245339      0.122670   
...                    ...       ...          ...           ...   
14d         6     0.002320  2.040153     0.249954      0.124977   
            7     0.003260  2.040153     0.249954      0.124977   
            9     0.002661  2.040153     0.249954      0.124977   
            10    0.001364  2.040153     0.249954      0.124977   
            15    0.003150  2.040153     0.249954      0.124977   

                precision@100    ndcg@5   ndcg@10  ndcg@100     map@5  \
window_size k                                                           
90d         15       0.012267  0.546162  0.546162  0.546162  0.872513   
            10       0.012267  0.546030  0.546030  0.546030  0.872297   
            8        0.012267  0.545987  0.545987  0.545987  0.872243   
            9        0.012267  0.545987  0.545987  0.545987  0.872243   
            7        0.012267  0.545811  0.545811  0.545811  0.871903   
...                       ...       ...       ...       ...       ...   
14d         6        0.012498  0.385844  0.385844  0.385844  0.783393   
            7        0.012498  0.385844  0.385844  0.385844  0.783393   
            9        0.012498  0.385844  0.385844  0.385844  0.783393   
            10       0.012498  0.385844  0.385844  0.385844  0.783393   
            15       0.012498  0.385844  0.385844  0.385844  0.783393   

                  map@10   map@100  recall@5  recall@10  recall@100  \
window_size k                                                         
90d         15  0.872513  0.872513       1.0        1.0         1.0   
            10  0.872297  0.872297       1.0        1.0         1.0   
            8   0.872243  0.872243       1.0        1.0         1.0   
            9   0.872243  0.872243       1.0        1.0         1.0   
            7   0.871903  0.871903       1.0        1.0         1.0   
...                  ...       ...       ...        ...         ...   
14d         6   0.783393  0.783393       1.0        1.0         1.0   
            7   0.783393  0.783393       1.0        1.0         1.0   
            9   0.783393  0.783393       1.0        1.0         1.0   
            10  0.783393  0.783393       1.0        1.0         1.0   
            15  0.783393  0.783393       1.0        1.0         1.0   

                r-precision@5  r-precision@10  r-precision@100  
window_size k                                                   
90d         15       0.964212        0.964212         0.964212  
            10       0.964212        0.964212         0.964212  
            8        0.964212        0.964212         0.964212  
            9        0.964212        0.964212         0.964212  
            7        0.964198        0.964198         0.964198  
...                       ...             ...              ...  
14d         6        0.971085        0.971085         0.971085  
            7        0.971085        0.971085         0.971085  
            9        0.971085        0.971085         0.971085  
            10       0.971085        0.971085         0.971085  
            15       0.971085        0.971085         0.971085  

[77 rows x 17 columns]

Now let's see the behaviour in each fold

In [10]:
best_avg_hparams = mdfi.set_index(['window_size', 'k']).loc[overall_hparams.iloc[0].name].reset_index().set_index(['fold', 'window_size', 'k'])
paths.save_model_results(best_avg_hparams, 'itemknn-best-avg', ORG_NAME, SPLITS_FREQ, SPLITS_NORMALIZE, K_RECOMMENDATIONS)
best_avg_hparams

Saved dataframe into /home/daviddavo/recsys4daos/data/output/Lido/models/itemknn-best-avg_W-WED_normalize.parquet


/tmp/ipykernel_3820567/3951593685.py:1: PerformanceWarning: indexing past lexsort depth may impact performance.
  best_avg_hparams = mdfi.set_index(['window_size', 'k']).loc[overall_hparams.iloc[0].name].reset_index().set_index(['fold', 'window_size', 'k'])


,,,fold_t,time_train,time_rec,open_proposals,min_recs,avg_recs,precision@1,precision@3,precision@5,precision@10,...,recall@10,recall@15,recall@100,r-precision@1,r-precision@3,r-precision@5,r-precision@10,r-precision@15,r-precision@100,time_eval
fold,window_size,k,,,,,,,,,,,,,,,,,,,,,
2022-05-04,90d,15,2022-05-04,0.001451,0.034660,1,1.0,1.000000,1.000000,0.333333,0.200000,0.100000,...,1.0,1.0,1.0,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.125006
2022-05-11,90d,15,2022-05-11,0.003701,0.083145,5,1.0,3.325581,0.430233,0.375969,0.313953,0.156977,...,1.0,1.0,1.0,0.540541,0.887500,0.900000,0.900000,0.900000,0.900000,0.137046
2022-05-18,90d,15,2022-05-18,0.001772,0.123761,1,1.0,1.000000,1.000000,0.333333,0.200000,0.100000,...,1.0,1.0,1.0,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.137551
2022-05-25,90d,15,2022-05-25,0.004131,0.184309,3,1.0,2.263682,0.597015,0.424544,0.254726,0.127363,...,1.0,1.0,1.0,0.847222,0.923358,0.923358,0.923358,0.923358,0.923358,0.157085
2022-06-01,90d,15,2022-06-01,0.002251,0.182306,2,1.0,1.502463,0.891626,0.359606,0.215764,0.107882,...,1.0,1.0,1.0,0.955801,1.000000,1.000000,1.000000,1.000000,1.000000,0.149846
2022-06-08,90d,15,2022-06-08,0.002846,0.275315,3,1.0,2.078947,0.769737,0.439693,0.263816,0.131908,...,1.0,1.0,1.0,0.799145,0.974684,0.974684,0.974684,0.974684,0.974684,0.164765
2022-06-15,90d,15,2022-06-15,0.002661,0.246675,2,1.0,1.714815,0.885185,0.344444,0.206667,0.103333,...,1.0,1.0,1.0,0.981172,1.000000,1.000000,1.000000,1.000000,1.000000,0.155623
2022-06-22,90d,15,2022-06-22,0.002788,0.217305,3,1.0,2.330544,0.732218,0.358438,0.215063,0.107531,...,1.0,1.0,1.0,0.970476,0.978022,0.978022,0.978022,0.978022,0.978022,0.155054
2022-06-29,90d,15,2022-06-29,0.003081,0.198804,3,1.0,2.527778,0.847222,0.356481,0.213889,0.106944,...,1.0,1.0,1.0,0.963570,0.986413,0.986413,0.986413,0.986413,0.986413,0.153263


### Best hparams by fold

This are the parameters used to check which model is the best (validation)

In [11]:
print("Best hyperparams by fold", OPTIM_METRIC)
best_hparams = mdfi.sort_values(OPTIM_METRIC,ascending=False).drop_duplicates(['fold'], keep='first').sort_values('fold').set_index(['fold', 'k', 'window_size'])
paths.save_model_results(best_hparams, 'itemknn-best-val', ORG_NAME, SPLITS_FREQ, SPLITS_NORMALIZE, K_RECOMMENDATIONS)
best_hparams[display_columns]

Best hyperparams by fold map@10
Saved dataframe into /home/daviddavo/recsys4daos/data/output/Lido/models/itemknn-best-val_W-WED_normalize.parquet


,,,time_train,avg_recs,precision@5,precision@10,precision@100,ndcg@5,ndcg@10,ndcg@100,map@5,map@10,map@100,recall@5,recall@10,recall@100,r-precision@5,r-precision@10,r-precision@100
fold,k,window_size,,,,,,,,,,,,,,,,,
2022-05-04,1,14d,0.000936,1.000000,0.200000,0.100000,0.010000,0.013333,0.013333,0.013333,1.000000,1.000000,1.000000,1.0,1.0,1.0,1.000000,1.000000,1.000000
2022-05-11,1,7d,0.005870,3.105263,0.321053,0.160526,0.016053,0.496314,0.496314,0.496314,0.621930,0.621930,0.621930,1.0,1.0,1.0,0.916667,0.916667,0.916667
2022-05-18,4,90d,0.001673,1.000000,0.200000,0.100000,0.010000,0.609524,0.609524,0.609524,1.000000,1.000000,1.000000,1.0,1.0,1.0,1.000000,1.000000,1.000000
2022-05-25,8,7d,0.001145,1.887218,0.270677,0.135338,0.013534,0.246399,0.246399,0.246399,0.866541,0.866541,0.866541,1.0,1.0,1.0,0.931193,0.931193,0.931193
2022-06-01,7,7d,0.001193,1.130841,0.209346,0.104673,0.010467,0.264072,0.264072,0.264072,0.985981,0.985981,0.985981,1.0,1.0,1.0,1.000000,1.000000,1.000000
2022-06-08,9,7d,0.001171,2.025381,0.282234,0.141117,0.014112,0.371274,0.371274,0.371274,0.893401,0.893401,0.893401,1.0,1.0,1.0,0.993631,0.993631,0.993631
2022-06-15,7,10YE,0.005000,1.715867,0.206642,0.103321,0.010332,0.613037,0.613037,0.613037,0.944649,0.944649,0.944649,1.0,1.0,1.0,1.000000,1.000000,1.000000
2022-06-22,8,90d,0.005554,2.330544,0.215063,0.107531,0.010753,0.557207,0.557207,0.557207,0.864365,0.864365,0.864365,1.0,1.0,1.0,0.978022,0.978022,0.978022
2022-06-29,15,90d,0.003081,2.527778,0.213889,0.106944,0.010694,0.591370,0.591370,0.591370,0.920139,0.920139,0.920139,1.0,1.0,1.0,0.986413,0.986413,0.986413


In [12]:
best_hparams.tail(len(best_hparams)-1).describe()

,fold_t,time_train,time_rec,open_proposals,min_recs,avg_recs,precision@1,precision@3,precision@5,precision@10,...,recall@10,recall@15,recall@100,r-precision@1,r-precision@3,r-precision@5,r-precision@10,r-precision@15,r-precision@100,time_eval
count,9,9.000000,9.000000,9.000000,9.0,9.000000,9.000000,9.000000,9.000000,9.000000,...,9.0,9.0,9.0,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000
mean,2022-06-08 00:00:00,0.003153,0.165469,2.777778,1.0,1.934370,0.809278,0.388549,0.242778,0.121389,...,1.0,1.0,1.0,0.873291,0.974854,0.976397,0.976397,0.976397,0.976397,0.148920
min,2022-05-11 00:00:00,0.001145,0.068629,1.000000,1.0,1.000000,0.434211,0.333333,0.200000,0.100000,...,1.0,1.0,1.0,0.525253,0.902778,0.916667,0.916667,0.916667,0.916667,0.133716
25%,2022-05-25 00:00:00,0.001193,0.116233,2.000000,1.0,1.686441,0.732218,0.348910,0.209346,0.104673,...,1.0,1.0,1.0,0.826389,0.978022,0.978022,0.978022,0.978022,0.978022,0.142462
50%,2022-06-08 00:00:00,0.003081,0.166799,3.000000,1.0,1.887218,0.847222,0.358438,0.215063,0.107531,...,1.0,1.0,1.0,0.963570,0.986413,0.986413,0.986413,0.986413,0.986413,0.153263
75%,2022-06-22 00:00:00,0.005000,0.217440,3.000000,1.0,2.330544,0.889831,0.443503,0.270677,0.135338,...,1.0,1.0,1.0,0.975962,1.000000,1.000000,1.000000,1.000000,1.000000,0.155018
max,2022-07-06 00:00:00,0.005870,0.255230,5.000000,1.0,3.105263,1.000000,0.470389,0.321053,0.160526,...,1.0,1.0,1.0,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.160011
std,NaN,0.001963,0.068431,1.092906,0.0,0.663184,0.170143,0.052608,0.043102,0.021551,...,0.0,0.0,0.0,0.156694,0.034534,0.031044,0.031044,0.031044,0.031044,0.009692


### Results of using best hparams in next fold

Kind of like the cvtt from [the LightGCN notebook](./11_microsoft_tuning.ipynb).

In [13]:
next_hparams_idx = best_hparams.index.to_frame(False)
next_hparams_idx['fold'] = next_hparams_idx['fold'].shift(-1)
next_hparams_idx = next_hparams_idx.dropna()
assert len(next_hparams_idx) == len(best_hparams)-1
next_hparams = mdfi.set_index(['fold', 'k', 'window_size']).loc[pd.MultiIndex.from_frame(next_hparams_idx)]
paths.save_model_results(next_hparams, 'itemknn-best-test', ORG_NAME, SPLITS_FREQ, SPLITS_NORMALIZE, K_RECOMMENDATIONS)
next_hparams[display_columns]

Saved dataframe into /home/daviddavo/recsys4daos/data/output/Lido/models/itemknn-best-test_W-WED_normalize.parquet


,,,time_train,avg_recs,precision@5,precision@10,precision@100,ndcg@5,ndcg@10,ndcg@100,map@5,map@10,map@100,recall@5,recall@10,recall@100,r-precision@5,r-precision@10,r-precision@100
fold,k,window_size,,,,,,,,,,,,,,,,,
2022-05-11,1,14d,0.001074,3.200000,0.320000,0.160000,0.016000,0.515635,0.515635,0.515635,0.608646,0.608646,0.608646,1.0,1.0,1.0,0.905405,0.905405,0.905405
2022-05-18,1,7d,0.004226,1.000000,0.200000,0.100000,0.010000,0.471429,0.471429,0.471429,1.000000,1.000000,1.000000,1.0,1.0,1.0,1.000000,1.000000,1.000000
2022-05-25,4,90d,0.002111,2.263682,0.254726,0.127363,0.012736,0.351303,0.351303,0.351303,0.797264,0.797264,0.797264,1.0,1.0,1.0,0.922794,0.922794,0.922794
2022-06-01,8,7d,0.001229,1.130841,0.209346,0.104673,0.010467,0.264072,0.264072,0.264072,0.985981,0.985981,0.985981,1.0,1.0,1.0,1.000000,1.000000,1.000000
2022-06-08,7,7d,0.003429,2.025381,0.282234,0.141117,0.014112,0.371274,0.371274,0.371274,0.893401,0.893401,0.893401,1.0,1.0,1.0,0.993631,0.993631,0.993631
2022-06-15,9,7d,0.001120,1.876712,0.205479,0.102740,0.010274,0.276445,0.276445,0.276445,0.732877,0.732877,0.732877,1.0,1.0,1.0,1.000000,1.000000,1.000000
2022-06-22,7,10YE,0.005314,2.338843,0.214876,0.107438,0.010744,0.561772,0.561772,0.561772,0.859160,0.859160,0.859160,1.0,1.0,1.0,0.978022,0.978022,0.978022
2022-06-29,8,90d,0.003045,2.527778,0.213889,0.106944,0.010694,0.591370,0.591370,0.591370,0.920139,0.920139,0.920139,1.0,1.0,1.0,0.986413,0.986413,0.986413
2022-07-06,15,90d,0.003273,2.348901,0.324176,0.162088,0.016209,0.588152,0.588152,0.588152,0.894918,0.894918,0.894918,1.0,1.0,1.0,0.915430,0.915430,0.915430


In [14]:
next_hparams.describe()

,fold_t,time_train,time_rec,open_proposals,min_recs,avg_recs,precision@1,precision@3,precision@5,precision@10,...,recall@10,recall@15,recall@100,r-precision@1,r-precision@3,r-precision@5,r-precision@10,r-precision@15,r-precision@100,time_eval
count,9,9.000000,9.000000,9.000000,9.0,9.000000,9.000000,9.000000,9.000000,9.000000,...,9.0,9.0,9.0,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000
mean,2022-06-08 00:00:00,0.002758,0.165667,2.777778,1.0,2.079126,0.732978,0.394394,0.247192,0.123596,...,1.0,1.0,1.0,0.860211,0.965354,0.966855,0.966855,0.966855,0.966855,0.151417
min,2022-05-11 00:00:00,0.001074,0.072240,1.000000,1.0,1.000000,0.425000,0.333333,0.200000,0.100000,...,1.0,1.0,1.0,0.524510,0.891892,0.905405,0.905405,0.905405,0.905405,0.133320
25%,2022-05-25 00:00:00,0.001229,0.095139,2.000000,1.0,1.876712,0.592040,0.348910,0.209346,0.104673,...,1.0,1.0,1.0,0.748818,0.922794,0.922794,0.922794,0.922794,0.922794,0.142014
50%,2022-06-08 00:00:00,0.003045,0.166548,3.000000,1.0,2.263682,0.774725,0.358127,0.214876,0.107438,...,1.0,1.0,1.0,0.963570,0.986413,0.986413,0.986413,0.986413,0.986413,0.154066
75%,2022-06-22 00:00:00,0.003429,0.199807,3.000000,1.0,2.348901,0.847222,0.424544,0.282234,0.141117,...,1.0,1.0,1.0,0.970588,1.000000,1.000000,1.000000,1.000000,1.000000,0.155688
max,2022-07-06 00:00:00,0.005314,0.334395,5.000000,1.0,3.200000,1.000000,0.540293,0.324176,0.162088,...,1.0,1.0,1.0,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.182368
std,NaN,0.001490,0.083143,1.092906,0.0,0.683699,0.204182,0.070350,0.050081,0.025041,...,0.0,0.0,0.0,0.160949,0.042873,0.040128,0.040128,0.040128,0.040128,0.014683


# User-based KNN

In [15]:
def testHParamsUserKNN(fold, k: int, window_size=None):
    # Create algorithm
    algo = user_knn.UserUser(
        feedback='implicit', # VERY IMPORTANT
        min_sim=0,
        # min_nbrs=0,
        nnbrs=k,  # the maximum number of neighbors for scoring each item (None for unlimited)
    )
    return test_with_hparams_lenskit(algo, all_folds[fold], K_RECOMMENDATIONS, window_size)

pd.Series(testHParamsUserKNN(last_folds_idx[-1],5,'14d'))

fold_t             2022-07-06 00:00:00
time_train                    0.592027
time_rec                      1.008992
open_proposals                       3
min_recs                             1
avg_recs                      2.411321
precision@1                   0.849057
precision@3                   0.545912
precision@5                   0.327547
precision@10                  0.163774
precision@15                  0.109182
precision@100                 0.016377
ndcg@1                        0.394046
ndcg@3                        0.427328
ndcg@5                        0.427328
ndcg@10                       0.427328
ndcg@15                       0.427328
ndcg@100                      0.427328
map@1                         0.849057
map@3                         0.878616
map@5                         0.878616
map@10                        0.878616
map@15                        0.878616
map@100                       0.878616
recall@1                      0.583019
recall@3                 

## Exploring hparams

In [16]:
results = explore_hparams(
    testHParamsUserKNN, 
    ParameterGrid({
        'fold': last_folds_idx,
        'k': ITEMKNN_Ks, # ,20,25,30,35,40,45,50,60,70,80,90,100],
        'window_size': WINDOW_SIZES,
    }),
    paths.hparams_progress('userknn', ORG_NAME, SPLITS_FREQ, SPLITS_NORMALIZE),
)
mdfu = pd.DataFrame(results)
mdfu

Restored checkpoint from ../.cache/Lido/hparams-userknn_W-WED_normalize.pkl with 847 results


  0%|          | 0/770 [00:00<?, ?it/s]

,fold,k,window_size,fold_t,time_train,time_rec,open_proposals,min_recs,avg_recs,precision@1,...,recall@10,recall@15,recall@100,r-precision@1,r-precision@3,r-precision@5,r-precision@10,r-precision@15,r-precision@100,time_eval
0,2022-05-04,1,7d,2022-05-04,0.000283,0.001157,1,NaN,NaN,0.000000,...,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.069558
1,2022-05-04,1,14d,2022-05-04,0.000363,0.002013,1,1.0,1.000000,1.000000,...,1.0,1.0,1.0,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.123368
2,2022-05-04,1,21d,2022-05-04,0.000387,0.020898,1,1.0,1.000000,1.000000,...,1.0,1.0,1.0,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.127244
3,2022-05-04,1,30d,2022-05-04,0.000550,0.023728,1,1.0,1.000000,1.000000,...,1.0,1.0,1.0,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.127859
4,2022-05-04,1,60d,2022-05-04,0.000579,0.024301,1,1.0,1.000000,1.000000,...,1.0,1.0,1.0,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.127226
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
765,2022-07-06,15,21d,2022-07-06,0.000930,0.208483,3,1.0,2.463333,0.830000,...,1.0,1.0,1.0,0.668005,0.988142,0.988142,0.988142,0.988142,0.988142,0.179374
766,2022-07-06,15,30d,2022-07-06,0.001151,0.225087,3,1.0,2.468750,0.731250,...,1.0,1.0,1.0,0.691595,0.919776,0.919776,0.919776,0.919776,0.919776,0.184898
767,2022-07-06,15,60d,2022-07-06,0.002174,0.253763,3,1.0,2.311047,0.764535,...,1.0,1.0,1.0,0.747782,0.906250,0.906250,0.906250,0.906250,0.906250,0.189128
768,2022-07-06,15,90d,2022-07-06,0.002510,0.268344,3,1.0,2.348901,0.780220,...,1.0,1.0,1.0,0.750587,0.910029,0.910029,0.910029,0.910029,0.910029,0.184632


### Best overall hparams

In [17]:
display_columns = ['time_train', 'avg_recs'] + [ c for c in mdfu.columns if c.endswith('@5') or c.endswith('@10') or c.endswith('@100') ]
overall_hparams = mdfu[mdfu['fold'] > last_folds_idx[0]].groupby(['window_size', 'k']).mean().sort_values(OPTIM_METRIC, ascending=False)
overall_hparams[display_columns]

time_train  avg_recs  precision@5  precision@10  \
window_size k                                                     
7d          2     0.000525  1.831533     0.242537      0.121268   
            1     0.000535  1.831533     0.242537      0.121268   
60d         1     0.001495  2.113616     0.245737      0.122869   
7d          3     0.000527  1.831533     0.242537      0.121268   
90d         1     0.001692  2.121412     0.245339      0.122670   
...                    ...       ...          ...           ...   
14d         8     0.000675  2.040153     0.249954      0.124977   
            7     0.000693  2.040153     0.249954      0.124977   
            9     0.000682  2.040153     0.249954      0.124977   
            10    0.000672  2.040153     0.249954      0.124977   
            15    0.000663  2.040153     0.249954      0.124977   

                precision@100    ndcg@5   ndcg@10  ndcg@100     map@5  \
window_size k                                                           
7d          2        0.012127  0.321568  0.321568  0.321568  0.870221   
            1        0.012127  0.321443  0.321443  0.321443  0.863415   
60d         1        0.012287  0.534832  0.534832  0.534832  0.857906   
7d          3        0.012127  0.316982  0.316982  0.316982  0.857810   
90d         1        0.012267  0.541498  0.541498  0.541498  0.857469   
...                       ...       ...       ...       ...       ...   
14d         8        0.012498  0.385118  0.385118  0.385118  0.780709   
            7        0.012498  0.385118  0.385118  0.385118  0.780709   
            9        0.012498  0.384746  0.384746  0.384746  0.779816   
            10       0.012498  0.384136  0.384136  0.384136  0.778417   
            15       0.012498  0.383990  0.383990  0.383990  0.777973   

                  map@10   map@100  recall@5  recall@10  recall@100  \
window_size k                                                         
7d          2   0.870221  0.870221       1.0        1.0         1.0   
            1   0.863415  0.863415       1.0        1.0         1.0   
60d         1   0.857906  0.857906       1.0        1.0         1.0   
7d          3   0.857810  0.857810       1.0        1.0         1.0   
90d         1   0.857469  0.857469       1.0        1.0         1.0   
...                  ...       ...       ...        ...         ...   
14d         8   0.780709  0.780709       1.0        1.0         1.0   
            7   0.780709  0.780709       1.0        1.0         1.0   
            9   0.779816  0.779816       1.0        1.0         1.0   
            10  0.778417  0.778417       1.0        1.0         1.0   
            15  0.777973  0.777973       1.0        1.0         1.0   

                r-precision@5  r-precision@10  r-precision@100  
window_size k                                                   
7d          2        0.975795        0.975795         0.975795  
            1        0.975305        0.975305         0.975305  
60d         1        0.960948        0.960948         0.960948  
7d          3        0.970968        0.970968         0.970968  
90d         1        0.961484        0.961484         0.961484  
...                       ...             ...              ...  
14d         8        0.970925        0.970925         0.970925  
            7        0.970925        0.970925         0.970925  
            9        0.970925        0.970925         0.970925  
            10       0.970925        0.970925         0.970925  
            15       0.970925        0.970925         0.970925  

[77 rows x 17 columns]

In [18]:
best_avg_hparams = mdfu.set_index(['window_size', 'k']).loc[overall_hparams.iloc[0].name].reset_index().set_index(['fold', 'window_size', 'k'])
paths.save_model_results(best_avg_hparams, 'userknn-best-avg', ORG_NAME, SPLITS_FREQ, SPLITS_NORMALIZE, K_RECOMMENDATIONS)
best_avg_hparams

Saved dataframe into /home/daviddavo/recsys4daos/data/output/Lido/models/userknn-best-avg_W-WED_normalize.parquet


/tmp/ipykernel_3820567/3337470013.py:1: PerformanceWarning: indexing past lexsort depth may impact performance.
  best_avg_hparams = mdfu.set_index(['window_size', 'k']).loc[overall_hparams.iloc[0].name].reset_index().set_index(['fold', 'window_size', 'k'])


,,,fold_t,time_train,time_rec,open_proposals,min_recs,avg_recs,precision@1,precision@3,precision@5,precision@10,...,recall@10,recall@15,recall@100,r-precision@1,r-precision@3,r-precision@5,r-precision@10,r-precision@15,r-precision@100,time_eval
fold,window_size,k,,,,,,,,,,,,,,,,,,,,,
2022-05-04,7d,2,2022-05-04,0.000295,0.001284,1,NaN,NaN,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.070423
2022-05-11,7d,2,2022-05-11,0.000528,0.051893,5,1.0,3.105263,0.592105,0.442982,0.321053,0.160526,...,1.0,1.0,1.0,0.651852,0.879630,0.888889,0.888889,0.888889,0.888889,0.133220
2022-05-18,7d,2,2022-05-18,0.000481,0.065754,1,1.0,1.000000,1.000000,0.333333,0.200000,0.100000,...,1.0,1.0,1.0,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.132328
2022-05-25,7d,2,2022-05-25,0.000507,0.089238,3,1.0,1.887218,0.902256,0.451128,0.270677,0.135338,...,1.0,1.0,1.0,0.815278,0.938525,0.938525,0.938525,0.938525,0.938525,0.149526
2022-06-01,7d,2,2022-06-01,0.000569,0.072197,2,1.0,1.130841,1.000000,0.348910,0.209346,0.104673,...,1.0,1.0,1.0,0.976636,1.000000,1.000000,1.000000,1.000000,1.000000,0.139897
2022-06-08,7d,2,2022-06-08,0.000585,0.132726,3,1.0,2.025381,0.796954,0.470389,0.282234,0.141117,...,1.0,1.0,1.0,0.742038,0.993631,0.993631,0.993631,0.993631,0.993631,0.156936
2022-06-15,7d,2,2022-06-15,0.000516,0.097752,2,1.0,1.876712,0.636986,0.342466,0.205479,0.102740,...,1.0,1.0,1.0,0.978495,1.000000,1.000000,1.000000,1.000000,1.000000,0.147062
2022-06-22,7d,2,2022-06-22,0.000502,0.100872,3,1.0,2.160000,0.360000,0.360000,0.216000,0.108000,...,1.0,1.0,1.0,0.895062,0.972222,0.972222,0.972222,0.972222,0.972222,0.146808
2022-06-29,7d,2,2022-06-29,0.000559,0.048527,3,1.0,1.611940,0.671642,0.353234,0.211940,0.105970,...,1.0,1.0,1.0,0.955556,0.988889,0.988889,0.988889,0.988889,0.988889,0.138414


### Best hparams by fold

In [19]:
print("Best hyperparams by fold", OPTIM_METRIC)
best_hparams = mdfu.sort_values(OPTIM_METRIC,ascending=False).drop_duplicates(['fold'], keep='first').sort_values('fold').set_index(['fold', 'k', 'window_size'])
paths.save_model_results(best_hparams, 'userknn-best-val', ORG_NAME, SPLITS_FREQ, SPLITS_NORMALIZE, K_RECOMMENDATIONS)
best_hparams[display_columns]

Best hyperparams by fold map@10
Saved dataframe into /home/daviddavo/recsys4daos/data/output/Lido/models/userknn-best-val_W-WED_normalize.parquet


,,,time_train,avg_recs,precision@5,precision@10,precision@100,ndcg@5,ndcg@10,ndcg@100,map@5,map@10,map@100,recall@5,recall@10,recall@100,r-precision@5,r-precision@10,r-precision@100
fold,k,window_size,,,,,,,,,,,,,,,,,
2022-05-04,1,14d,0.000363,1.000000,0.200000,0.100000,0.010000,0.013333,0.013333,0.013333,1.000000,1.000000,1.000000,1.0,1.0,1.0,1.000000,1.000000,1.000000
2022-05-11,1,7d,0.000598,3.105263,0.321053,0.160526,0.016053,0.599964,0.599964,0.599964,0.815058,0.815058,0.815058,1.0,1.0,1.0,0.906061,0.906061,0.906061
2022-05-18,1,10YE,0.001719,1.000000,0.200000,0.100000,0.010000,0.619048,0.619048,0.619048,1.000000,1.000000,1.000000,1.0,1.0,1.0,1.000000,1.000000,1.000000
2022-05-25,4,7d,0.000528,1.887218,0.270677,0.135338,0.013534,0.261362,0.261362,0.261362,0.936090,0.936090,0.936090,1.0,1.0,1.0,0.938525,0.938525,0.938525
2022-06-01,3,7d,0.000615,1.130841,0.209346,0.104673,0.010467,0.266833,0.266833,0.266833,1.000000,1.000000,1.000000,1.0,1.0,1.0,1.000000,1.000000,1.000000
2022-06-08,15,7d,0.000496,2.025381,0.282234,0.141117,0.014112,0.376557,0.376557,0.376557,0.911168,0.911168,0.911168,1.0,1.0,1.0,0.993902,0.993902,0.993902
2022-06-15,9,90d,0.001890,1.714815,0.206667,0.103333,0.010333,0.611549,0.611549,0.611549,0.946296,0.946296,0.946296,1.0,1.0,1.0,1.000000,1.000000,1.000000
2022-06-22,15,30d,0.001230,2.153439,0.217989,0.108995,0.010899,0.434745,0.434745,0.434745,0.847443,0.847443,0.847443,1.0,1.0,1.0,0.967626,0.967626,0.967626
2022-06-29,15,10YE,0.003218,2.529954,0.213825,0.106912,0.010691,0.570891,0.570891,0.570891,0.870968,0.870968,0.870968,1.0,1.0,1.0,0.982249,0.982249,0.982249


In [20]:
best_hparams.tail(len(best_hparams)-1).describe()[display_columns]

,time_train,avg_recs,precision@5,precision@10,precision@100,ndcg@5,ndcg@10,ndcg@100,map@5,map@10,map@100,recall@5,recall@10,recall@100,r-precision@5,r-precision@10,r-precision@100
count,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.0,9.0,9.0,9.000000,9.000000,9.000000
mean,0.001208,1.914817,0.243099,0.121550,0.012155,0.437589,0.437589,0.437589,0.918476,0.918476,0.918476,1.0,1.0,1.0,0.974427,0.974427,0.974427
min,0.000496,1.000000,0.200000,0.100000,0.010000,0.197356,0.197356,0.197356,0.815058,0.815058,0.815058,1.0,1.0,1.0,0.906061,0.906061,0.906061
25%,0.000580,1.686441,0.209346,0.104673,0.010467,0.266833,0.266833,0.266833,0.870968,0.870968,0.870968,1.0,1.0,1.0,0.967626,0.967626,0.967626
50%,0.000615,1.887218,0.217989,0.108995,0.010899,0.434745,0.434745,0.434745,0.936090,0.936090,0.936090,1.0,1.0,1.0,0.982249,0.982249,0.982249
75%,0.001719,2.153439,0.270677,0.135338,0.013534,0.599964,0.599964,0.599964,0.946296,0.946296,0.946296,1.0,1.0,1.0,1.000000,1.000000,1.000000
max,0.003218,3.105263,0.321053,0.160526,0.016053,0.619048,0.619048,0.619048,1.000000,1.000000,1.000000,1.0,1.0,1.0,1.000000,1.000000,1.000000
std,0.000926,0.652797,0.042880,0.021440,0.002144,0.169183,0.169183,0.169183,0.064090,0.064090,0.064090,0.0,0.0,0.0,0.032522,0.032522,0.032522


### Results of using best hparams in next fold

Kind of like the cvtt from [the LightGCN notebook](./11_microsoft_tuning.ipynb)

In [21]:
next_hparams_idx = best_hparams.index.to_frame(False)
next_hparams_idx['fold'] = next_hparams_idx['fold'].shift(-1)
next_hparams_idx = next_hparams_idx.dropna()
assert len(next_hparams_idx) == len(best_hparams)-1
next_hparams = mdfu.set_index(['fold', 'k', 'window_size']).loc[pd.MultiIndex.from_frame(next_hparams_idx)]
paths.save_model_results(next_hparams, 'userknn-best-test', ORG_NAME, SPLITS_FREQ, SPLITS_NORMALIZE, K_RECOMMENDATIONS)
next_hparams

Saved dataframe into /home/daviddavo/recsys4daos/data/output/Lido/models/userknn-best-test_W-WED_normalize.parquet


,,,fold_t,time_train,time_rec,open_proposals,min_recs,avg_recs,precision@1,precision@3,precision@5,precision@10,...,recall@10,recall@15,recall@100,r-precision@1,r-precision@3,r-precision@5,r-precision@10,r-precision@15,r-precision@100,time_eval
fold,k,window_size,,,,,,,,,,,,,,,,,,,,,
2022-05-11,1,14d,2022-05-11,0.000659,0.054532,5,1.0,3.200000,0.700000,0.483333,0.320000,0.160000,...,1.0,1.0,1.0,0.633929,0.887931,0.896552,0.896552,0.896552,0.896552,0.133981
2022-05-18,1,7d,2022-05-18,0.000402,0.065036,1,1.0,1.000000,1.000000,0.333333,0.200000,0.100000,...,1.0,1.0,1.0,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.131655
2022-05-25,1,10YE,2022-05-25,0.001951,0.147496,3,1.0,2.278049,0.678049,0.422764,0.253659,0.126829,...,1.0,1.0,1.0,0.846523,0.930000,0.930000,0.930000,0.930000,0.930000,0.156572
2022-06-01,4,7d,2022-06-01,0.000557,0.073881,2,1.0,1.130841,1.000000,0.348910,0.209346,0.104673,...,1.0,1.0,1.0,0.976636,1.000000,1.000000,1.000000,1.000000,1.000000,0.142477
2022-06-08,3,7d,2022-06-08,0.000598,0.131287,3,1.0,2.025381,0.796954,0.470389,0.282234,0.141117,...,1.0,1.0,1.0,0.742038,0.993631,0.993631,0.993631,0.993631,0.993631,0.156045
2022-06-15,15,7d,2022-06-15,0.000510,0.098317,2,1.0,1.876712,0.500000,0.342466,0.205479,0.102740,...,1.0,1.0,1.0,0.972603,1.000000,1.000000,1.000000,1.000000,1.000000,0.146291
2022-06-22,9,90d,2022-06-22,0.001992,0.168091,3,1.0,2.330544,0.652720,0.358438,0.215063,0.107531,...,1.0,1.0,1.0,0.960470,0.975155,0.975155,0.975155,0.975155,0.975155,0.154892
2022-06-29,15,30d,2022-06-29,0.001162,0.121742,3,1.0,2.444444,0.321637,0.348928,0.209357,0.104678,...,1.0,1.0,1.0,0.951515,0.973684,0.973684,0.973684,0.973684,0.973684,0.148380
2022-07-06,15,10YE,2022-07-06,0.003305,0.266650,3,1.0,2.359459,0.778378,0.537838,0.322703,0.161351,...,1.0,1.0,1.0,0.752315,0.911079,0.911079,0.911079,0.911079,0.911079,0.181889


In [22]:
next_hparams.describe()

,fold_t,time_train,time_rec,open_proposals,min_recs,avg_recs,precision@1,precision@3,precision@5,precision@10,...,recall@10,recall@15,recall@100,r-precision@1,r-precision@3,r-precision@5,r-precision@10,r-precision@15,r-precision@100,time_eval
count,9,9.000000,9.000000,9.000000,9.0,9.000000,9.000000,9.000000,9.000000,9.000000,...,9.0,9.0,9.0,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000
mean,2022-06-08 00:00:00,0.001237,0.125226,2.777778,1.0,2.071715,0.714193,0.405155,0.246427,0.123213,...,1.0,1.0,1.0,0.870670,0.963498,0.964456,0.964456,0.964456,0.964456,0.150242
min,2022-05-11 00:00:00,0.000402,0.054532,1.000000,1.0,1.000000,0.321637,0.333333,0.200000,0.100000,...,1.0,1.0,1.0,0.633929,0.887931,0.896552,0.896552,0.896552,0.896552,0.131655
25%,2022-05-25 00:00:00,0.000557,0.073881,2.000000,1.0,1.876712,0.652720,0.348910,0.209346,0.104673,...,1.0,1.0,1.0,0.752315,0.930000,0.930000,0.930000,0.930000,0.930000,0.142477
50%,2022-06-08 00:00:00,0.000659,0.121742,3.000000,1.0,2.278049,0.700000,0.358438,0.215063,0.107531,...,1.0,1.0,1.0,0.951515,0.975155,0.975155,0.975155,0.975155,0.975155,0.148380
75%,2022-06-22 00:00:00,0.001951,0.147496,3.000000,1.0,2.359459,0.796954,0.470389,0.282234,0.141117,...,1.0,1.0,1.0,0.972603,1.000000,1.000000,1.000000,1.000000,1.000000,0.156045
max,2022-07-06 00:00:00,0.003305,0.266650,5.000000,1.0,3.200000,1.000000,0.537838,0.322703,0.161351,...,1.0,1.0,1.0,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.181889
std,NaN,0.000987,0.065540,1.092906,0.0,0.678077,0.217832,0.075728,0.050163,0.025081,...,0.0,0.0,0.0,0.132348,0.042903,0.041061,0.041061,0.041061,0.041061,0.014942
